In [1]:
import torch
from torch import nn, optim
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms

class CNN(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.fc_layers = nn.Sequential(
            nn.Linear(64 * 8 * 8, 500),
            nn.ReLU(),
            nn.Linear(500, num_classes)
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = torch.flatten(x, 1)  # Flatten the output for the fully connected layers
        return self.fc_layers(x)

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))  # Normalize the images
])

dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform) #Load dataset


'''
Split dataset into training and validation
'''

train_size = int(0.8 * len(dataset))  # Determine size of training set (80% of total)
val_size = len(dataset) - train_size # The remaining for validation
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64)

model = CNN(num_classes=10)
loss_fn = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

'''
Function to train the model
'''
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train() # Set the model to training mode
    for batch, (X, y) in enumerate(dataloader):
        pred = model(X) # Make predictions
        y_one_hot = nn.functional.one_hot(y, num_classes=10).float()
        loss = loss_fn(pred, y_one_hot)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

'''
Function to evaluate the model
'''
def validate(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    val_loss, correct = 0, 0
    with torch.no_grad(): # Disable gradient computation
        for X, y in dataloader:
            pred = model(X) # Make predictions
            y_one_hot = nn.functional.one_hot(y, num_classes=10).float()
            val_loss += loss_fn(pred, y_one_hot).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    val_loss /= num_batches
    accuracy = correct / size
    print(f"Validation Error: \n Avg loss: {val_loss:>8f} \n Accuracy: {accuracy:>8f}")


epochs = 10
for epoch in range(epochs):
    print(f"Epoch {epoch+1}\n-------------------------------")
    train(train_loader, model, loss_fn, optimizer)
    validate(val_loader, model, loss_fn)
print("Training is finished")

test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=64)

validate(test_loader, model, loss_fn) # test



100%|██████████| 170498071/170498071 [00:02<00:00, 78794671.65it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Epoch 1
-------------------------------
loss: 0.105285  [    0/40000]
loss: 0.075808  [ 6400/40000]
loss: 0.074145  [12800/40000]
loss: 0.061794  [19200/40000]
loss: 0.056761  [25600/40000]
loss: 0.063771  [32000/40000]
loss: 0.049783  [38400/40000]
Validation Error: 
 Avg loss: 0.055908 
 Accuracy: 0.608000
Epoch 2
-------------------------------
loss: 0.059978  [    0/40000]
loss: 0.064106  [ 6400/40000]
loss: 0.058061  [12800/40000]
loss: 0.047409  [19200/40000]
loss: 0.053049  [25600/40000]
loss: 0.040082  [32000/40000]
loss: 0.046481  [38400/40000]
Validation Error: 
 Avg loss: 0.051056 
 Accuracy: 0.654600
Epoch 3
-------------------------------
loss: 0.043007  [    0/40000]
loss: 0.050673  [ 6400/40000]
loss: 0.038573  [12800/40000]
loss: 0.036789  [19200/40000]
loss: 0.040519  [25600/40000]
loss: 0.050735  [32000/40000]
loss: 0.039312  [38400/40000]
Validation Error: 
 Avg loss: 0.045885 
 Accuracy: 0.700800
Epoch 4
-----------